#**LDA Topic Modelling and Experiments**

#### Importing required libraries

In [1]:
import gensim

In [2]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 57.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [ ]:
!pip install gensim

In [3]:
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora
from gensim.models import TfidfModel
import pyLDAvis
import pyLDAvis.gensim
import pandas as pd
import nltk
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder, QuadgramAssocMeasures, QuadgramCollocationFinder
from nltk import word_tokenize

In [4]:
nltk.download('words')
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### **N-grams, TF-IDF, LDA** (and more pre-processing)

In [5]:
class topic_modeling:
    def __init__(self, facts):
        self.facts = facts

    def organise(self):
        facts_words = []
        for fact in self.facts:
            split_fact = fact.split()
            facts_words.append(split_fact)
        return facts_words

    def remove_lang_words(self,facts_words, english_words):

      # Filtering out non-english words and three letter words
      filtered = [word for word in facts_words if word in english_words and len(word)>3]
      return filtered

    def find_bigrams(self, facts):
      text = ' '.join(facts)
      tokens = word_tokenize(text)
      bigrams = list(nltk.bigrams(tokens))
      return bigrams

    def view_frequencies(self, id2word, texts, corpus, tfidf):
      word_frequency = []
      for doc in corpus:
          # Convert the document to TF-IDF representation
          tfidf_vector = tfidf[doc]

          # Iterate over the TF-IDF values for each word in the document
          for word_id, tfidf_score in tfidf_vector:
              # Get the word corresponding to the word ID
              word = id2word[word_id]

              word_frequency.append([word,tfidf_score])

      df_freq = pd.DataFrame(word_frequency, columns = ['Word', 'Frequency'])
      df_sorted = df_freq.sort_values('Frequency', ascending=False)

      return df_sorted

    def filter_tf_idf(self, id2word, texts, corpus, tfidf):

        low_value = 0.03
        high_value = 0.6
        words = []
        words_missing_in_tfidf = []
        for i in range(0, len(corpus)):
            bow = corpus[i]
            value_words = []  # reinitialize to be safe. You can skip this.
            tfidf_ids = [id for id, value in tfidf[bow]]
            bow_ids = [id for id, value in bow]
            value_words = [id for id, value in tfidf[bow] if value < low_value or value > high_value]
            drops = value_words + words_missing_in_tfidf

            for item in drops:
                words.append(id2word[item])
            words_missing_in_tfidf = [id for id in bow_ids if
                                      id not in tfidf_ids]  # The words with tf-idf socre 0 will be missing

            new_bow = [b for b in bow if b[0] not in value_words and b[0] not in words_missing_in_tfidf]
            corpus[i] = new_bow

        return corpus, id2word

    def perform_lda(self, corpus, id2word):
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                    id2word=id2word,
                                                    num_topics=10,
                                                    random_state=100,
                                                    update_every=1,
                                                    chunksize=50,
                                                    passes=10,
                                                    alpha="auto")

        return lda_model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Violation Cases

#### Importing facts corresponding to violation cases

In [6]:
from google.colab import files
import io

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Import Processed Facts

In [ ]:
uploaded_proc = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving violation_facts_processed.csv to violation_facts_processed.csv


In [ ]:
violation_tm = pd.read_csv(io.BytesIO(uploaded_proc['violation_facts_processed.csv']))

### Import Lemmatized Facts

In [ ]:
uploaded_lemma = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving violation_facts_lemma.csv to violation_facts_lemma (1).csv


In [ ]:
violation_tm = pd.read_csv(io.BytesIO(uploaded_lemma['violation_facts_lemma.csv']))

### Import Stop Word Removal Facts

In [ ]:
uploaded_sw = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving violation_facts_sw.csv to violation_facts_sw (1).csv


In [ ]:
violation_tm = pd.read_csv(io.BytesIO(uploaded_sw['violation_facts_sw.csv']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Import No Preprocessing Facts

In [ ]:
uploaded_none = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving violation_facts_none.csv to violation_facts_none (1).csv


In [ ]:
violation_tm = pd.read_csv(io.BytesIO(uploaded_none['violation_facts_none.csv']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Non Violation

### Processed

In [7]:
nv_proc = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving non_violation_facts_processed.csv to non_violation_facts_processed.csv


In [8]:
violation_tm = pd.read_csv(io.BytesIO(nv_proc['non_violation_facts_processed.csv']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Lemma

In [26]:
nv_lemma = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving non_violation_facts_lemma.csv to non_violation_facts_lemma.csv


In [27]:
violation_tm = pd.read_csv(io.BytesIO(nv_lemma['non_violation_facts_lemma.csv']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### SW Removal

In [43]:
nv_sw = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving non_violation_facts_sw.csv to non_violation_facts_sw.csv


In [44]:
violation_tm = pd.read_csv(io.BytesIO(nv_sw['non_violation_facts_sw.csv']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### No Preprocessing

In [60]:
nv_np = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving non_violation_facts_none.csv to non_violation_facts_none.csv


In [61]:
violation_tm = pd.read_csv(io.BytesIO(nv_np['non_violation_facts_none.csv']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [62]:
violation_tm.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Facts,Cluster,x0,x1
0,0,in and lives in kragujevac at the relevant tim...,NaN,NaN,NaN
1,1,the applicant was born in and lives in weinhei...,NaN,NaN,NaN
2,2,the applicant company is a public limited comp...,NaN,NaN,NaN
3,3,is a newspaper published in baku it has legal ...,NaN,NaN,NaN
4,4,the applicant was born in and lives in warsaw ...,NaN,NaN,NaN


#### Some more pre-processing

In [69]:
# Splitting into a list of lists of words
facts = violation_tm['Facts'].tolist()
tm = topic_modeling(facts)
facts_words = tm.organise()
print(len(facts_words))

73


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [70]:
# Removing non english words
final_data = []
english_words = set(nltk.corpus.words.words())
for i in range(0, len(facts_words)):
  facts_words_filtered = tm.remove_lang_words(facts_words[i], english_words)
  final_data.append(facts_words_filtered)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
print(final_data[0])

['life', 'relevant', 'time', 'journalist', 'weekly', 'news', 'magazine', 'based', 'town', 'agent', 'party', 'titled', 'shame', 'silence', 'piece', 'written', 'relevant', 'reason', 'ashamed', 'towards', 'head', 'municipal', 'branch', 'office', 'firstly', 'city', 'front', 'main', 'post', 'office', 'building', 'falsely', 'street', 'vendor', 'market', 'inspector', 'pretext', 'certain', 'document', 'copied', 'thereafter', 'drove', 'girl', 'away', 'street', 'tried', 'rape', 'girl', 'origin', 'resistance', 'scream', 'ultimately', 'assailant', 'girl', 'tell', 'anyone', 'give', 'telephone', 'number', 'next', 'head', 'municipal', 'branch', 'office', 'meet', 'also', 'period', 'hour', 'brought', 'investigating', 'assailant', 'false', 'impersonation', 'unlawful', 'deprivation', 'liberty', 'rape', 'shall', 'course', 'conclusion', 'local', 'otherwise', 'prompt', 'come', 'even', 'much', 'lesser', 'simply', 'kept', 'silent', 'incident', 'press', 'also', 'received', 'information', 'investigating', 'pros

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Creating bigrams

In [71]:
bigram_list = []
for data in final_data:
  bigrams = tm.find_bigrams(data)
  bigram_list.append(bigrams)

print(bigram_list[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('relevant', 'time'), ('time', 'journalist'), ('journalist', 'weekly'), ('weekly', 'news'), ('news', 'magazine'), ('magazine', 'based'), ('based', 'same'), ('same', 'town'), ('town', 'applicant'), ('applicant', 'lawyer'), ('lawyer', 'government'), ('government', 'were'), ('were', 'their'), ('their', 'agent'), ('agent', 'case'), ('case', 'june'), ('june', 'article'), ('article', 'titled'), ('titled', 'shame'), ('shame', 'through'), ('through', 'silence'), ('silence', 'this'), ('this', 'piece'), ('piece', 'written'), ('written', 'applicant'), ('applicant', 'relevant'), ('relevant', 'reason'), ('reason', 'ashamed'), ('ashamed', 'towards'), ('towards', 'head'), ('head', 'municipal'), ('municipal', 'branch'), ('branch', 'office'), ('office', 'firstly'), ('firstly', 'city'), ('city', 'front'), ('front', 'main'), ('main', 'post'), ('post', 'office'), ('office', 'building'), ('building', 'falsely'), ('falsely', 'himself'), ('himself', 'street'), ('street', 'vendor'), ('vendor', 'market'), ('m

In [72]:
combined_tuples = []
final_bigram_list = []
for bigrams in bigram_list:
  combined_tuples = []
  for tuples in bigrams:
    combined = '_'.join(tuples)
    combined_tuples.append(combined)
  final_bigram_list.append(combined_tuples)
print(final_bigram_list[1])
final_data = final_bigram_list

['applicant_born', 'born_campaigner', 'campaigner_against', 'against_abortion', 'abortion_applicant', 'applicant_distributed', 'distributed_immediate', 'immediate_vicinity', 'vicinity_medical', 'medical_practice', 'practice_inter', 'inter_following', 'following_text', 'text_front', 'front_page', 'page_know', 'know_that', 'that_full', 'full_name', 'name_address', 'address_that', 'that_unlawful', 'unlawful_according', 'according_federal', 'federal_constitutional', 'constitutional_court', 'court_nach', 'nach_sind', 'sind_underneath', 'underneath_following', 'following_smaller', 'smaller_type', 'type_according', 'according_international', 'international_criminal', 'criminal_murder', 'murder_intentional', 'intentional_innocent', 'innocent_human', 'human_being', 'being_tode', 'tode_back', 'back_side', 'side_folded', 'folded_leaflet', 'leaflet_following', 'following_text', 'text_murder', 'murder_human', 'human_unlawful', 'unlawful_morally', 'morally_degraded', 'degraded_state', 'state_murder'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### TFIDF and LDA

In [73]:
id2word = corpora.Dictionary(final_data)
texts = final_data
corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(corpus, id2word=id2word)

df_sorted = tm.view_frequencies(id2word, texts, corpus, tfidf)
df_sorted

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word,Frequency
84411,university_hospital,0.679984
149817,anonymous_letter,0.648767
163289,military_service,0.592997
147408,applicant_association,0.515443
69871,prime_minister,0.499757
...,...,...
45993,present_case,0.000056
45939,fact_that,0.000056
129446,interference_with,0.000055
157021,interference_with,0.000051


In [74]:
corpus_filter, id2word_filter = tm.filter_tf_idf(id2word, texts, corpus, tfidf)
lda_model = tm.perform_lda(corpus_filter, id2word_filter)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Visualisation

Processed

In [24]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_filter, id2word_filter, mds="mmds", R=30)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.126356 -0.081723       1        1  19.956541
1      0.016955  0.119387       2        1  15.273004
9     -0.082085  0.064202       3        1  15.041572
5      0.043340 -0.086074       4        1  12.627142
2      0.094333 -0.011987       5        1  10.900403
7      0.048646  0.017382       6        1   6.348753
6      0.011873  0.025666       7        1   5.954619
0     -0.007629 -0.035895       8        1   5.768164
3     -0.009949 -0.000472       9        1   5.510437
4      0.010871 -0.010486      10        1   2.619366, topic_info=                         Term       Freq      Total Category  logprob  loglift
10453           personal_data  21.000000  21.000000  Default  30.0000  30.0000
4161              hate_speech  32.000000  32.000000  Default  29.0000  29.0000
29854        radio_television  27.000000  27.000000  Default  28.0000  28.0000
46113          state_security  23.000000  23.000000  Default  27.0000  27.0000
9761     individual_concerned  22.000000  22.000000  Default  26.0000  26.0000
...                       ...        ...        ...      ...      ...      ...
94566               year_rank   0.607305   0.809689  Topic10  -7.1468   3.3546
41133            tomb_unknown   0.578780   0.781309  Topic10  -7.1949   3.3422
39641        desecration_tomb   0.578778   0.781316  Topic10  -7.1949   3.3422
64489           remove_banner   0.497063   0.699514  Topic10  -7.3471   3.3006
48337  discharge_professional   0.482927   0.685312  Topic10  -7.3760   3.2922

[386 rows x 6 columns], token_table=       Topic      Freq                  Term
term                                        
1911       3  0.866167       abortion_doctor
1921       3  1.002916       abortion_murder
7359       8  0.810070      abuse_individual
87256      1  1.023082           abuse_trust
5          5  0.820976  acceptable_criticism
...      ...       ...                   ...
55414      9  1.392705            weak_woman
28306      3  0.894978          wear_clothes
58436      5  1.110664        weigh_interest
11456      8  0.699092            whole_part
94566     10  1.235042             year_rank

[387 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 10, 6, 3, 8, 7, 1, 4, 5])

Lemmatized

In [41]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_filter, id2word_filter, mds="mmds", R=30)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.071995  0.115539       1        1  18.063118
7      0.008691 -0.143398       2        1  17.626772
1     -0.132114 -0.033826       3        1  15.821390
5     -0.054963  0.074504       4        1  10.070693
3      0.029762 -0.038905       5        1   9.778244
0      0.110545 -0.005928       6        1   9.206677
8     -0.001334  0.033130       7        1   8.063059
9     -0.007818 -0.009263       8        1   4.122035
2     -0.029968  0.004335       9        1   3.984611
4      0.005202  0.003813      10        1   3.263402, topic_info=                       Term        Freq       Total Category  logprob  loglift
4045      applicant_company  223.000000  223.000000  Default  30.0000  30.0000
602            court_appeal   65.000000   65.000000  Default  29.0000  29.0000
10844  administrative_court   69.000000   69.000000  Default  28.0000  28.0000
92599      military_service   35.000000   35.000000  Default  27.0000  27.0000
29443         search_engine   41.000000   41.000000  Default  26.0000  26.0000
...                     ...         ...         ...      ...      ...      ...
17156             that_book    0.344925    1.365315  Topic10  -8.1050   2.0466
40275             copy_book    0.344949    1.526514  Topic10  -8.1050   1.9351
12800     length_proceeding    0.344939    1.581822  Topic10  -8.1050   1.8994
8538         district_court    0.400245   17.717203  Topic10  -7.9563  -0.3678
21478            penal_code    0.400917   20.034544  Topic10  -7.9546  -0.4891

[449 rows x 6 columns], token_table=       Topic      Freq               Term
term                                     
2690       2  0.458601    abortion_murder
2690       5  0.458601    abortion_murder
66248      2  0.968506       abuse_public
15116      9  0.796470  access_classified
15117      9  0.887856    access_document
...      ...       ...                ...
93349      6  0.955240      with_ministry
3799       2  0.966920        with_regard
52860      7  0.730729     withdrawn_from
10695      8  0.679083     without_notice
90835      4  1.056687  worked_prosecutor

[484 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 2, 6, 4, 1, 9, 10, 3, 5])

Only Stop Word Removal

In [58]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_filter, id2word_filter, mds="mmds", R=30)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.108086 -0.066735       1        1  19.900101
4      0.139679  0.029373       2        1  17.167719
9     -0.081070  0.093616       3        1  16.256832
1      0.030547 -0.088146       4        1  10.366192
7      0.039700 -0.025010       5        1   8.065765
0      0.033656  0.047580       6        1   7.707722
5     -0.010677  0.047378       7        1   6.884769
8     -0.000810 -0.017846       8        1   6.508481
2     -0.041399 -0.020343       9        1   6.272155
3     -0.001540  0.000133      10        1   0.870264, topic_info=                           Term       Freq      Total Category  logprob  \
9422              personal_data  48.000000  48.000000  Default  30.0000   
2510         access_information  50.000000  50.000000  Default  29.0000   
3716                hate_speech  40.000000  40.000000  Default  28.0000   
4830           radio_television  18.000000  18.000000  Default  27.0000   
22139             search_engine  24.000000  24.000000  Default  26.0000   
...                         ...        ...        ...      ...      ...   
3863          information_shall   0.048145   1.111283  Topic10  -8.5444   
61406              upon_receipt   0.048145   1.443987  Topic10  -8.5444   
2510         access_information   0.056852  50.592288  Topic10  -8.3781   
12338  significant_disadvantage   0.043772   7.510733  Topic10  -8.6396   
6044         interference_first   0.032928   0.615091  Topic10  -8.9243   

       loglift  
9422   30.0000  
2510   29.0000  
3716   28.0000  
4830   27.0000  
22139  26.0000  
...        ...  
3863    1.6051  
61406   1.3432  
2510   -2.0470  
12338  -0.4010  
6044    1.8167  

[395 rows x 6 columns], token_table=       Topic      Freq                  Term
term                                        
79656      4  0.899228           abuse_trust
5          2  0.946759  acceptable_criticism
2510       1  0.889464    access_information
2510       3  0.079063    access_information
2510       5  0.019766    access_information
...      ...       ...                   ...
95471      5  0.805428          whose_source
2475       7  0.796650        within_context
76724      5  1.157473     without_knowledge
7472       4  1.086600        without_notice
29230      6  1.098097          would_longer

[388 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 10, 2, 8, 1, 6, 9, 3, 4])

No Pre-processing

In [75]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_filter, id2word_filter, mds="mmds", R=30)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.129723  0.012025       1        1  14.916006
2      0.042308  0.136889       2        1  14.841888
1     -0.106332 -0.042022       3        1  12.739363
9     -0.071814  0.067022       4        1  12.328962
4      0.042248 -0.084042       5        1   9.887504
8     -0.035537  0.028699       6        1   9.861305
6     -0.023709 -0.092524       7        1   8.954633
0     -0.013912 -0.013794       8        1   6.917953
3      0.032630 -0.010464       9        1   6.575926
5      0.004395 -0.001790      10        1   2.976460, topic_info=                       Term       Freq      Total Category  logprob  loglift
3738      applicant_company  99.000000  99.000000  Default  30.0000  30.0000
9996   administrative_court  70.000000  70.000000  Default  29.0000  29.0000
86289      military_service  50.000000  50.000000  Default  28.0000  28.0000
539            court_appeal  53.000000  53.000000  Default  27.0000  27.0000
4981            hate_speech  33.000000  33.000000  Default  26.0000  26.0000
...                     ...        ...        ...      ...      ...      ...
6713          supreme_court   0.723567  33.428645  Topic10  -7.2003  -0.3185
963              high_court   0.467987   5.308709  Topic10  -7.6361   1.0858
4928    general_information   0.314741   1.178780  Topic10  -8.0328   2.1940
544             court_first   0.314742   5.726582  Topic10  -8.0328   0.6133
812          first_instance   0.314748   6.418395  Topic10  -8.0328   0.4993

[444 rows x 6 columns], token_table=       Topic      Freq                  Term
term                                        
48134      4  0.867410            about_book
90777      2  1.033120           abuse_trust
44         4  1.061419  acceptable_criticism
3559       1  0.067198    access_information
3559       2  0.604780    access_information
...      ...       ...                   ...
65126      4  0.814626        with_editorial
87012      2  0.960785         with_ministry
3500       1  0.964166           with_regard
21684      7  0.821861       without_consent
84659      6  1.053941     worked_prosecutor

[471 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 2, 10, 5, 9, 7, 1, 4, 6])

In [76]:
# Dataframe with first 5 words of each topic

topics_df = pd.DataFrame(columns=['Topic', 'Top 5 Terms', 'Prob'])
results = lda_model.show_topics(num_topics=10, num_words=5, log=False, formatted=False)

for i in range(0, 10):
    terms_five = []
    prob_five = []
    for j in range(0, 5):
        terms_five.append(results[i][1][j][0])
        prob_five.append(results[i][1][j][1])

    new_row = pd.DataFrame({'Topic': [i], 'Top 5 Terms': [terms_five], 'Prob': [prob_five]})
    topics_df = pd.concat([topics_df, new_row], ignore_index=True)

topics_df.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic,Top 5 Terms,Prob
0,0,"[second_applicant, first_applicant, editorial_...","[0.0025694198, 0.0025201715, 0.002372393, 0.00..."
1,1,"[hate_speech, court_cassation, wall_account, a...","[0.00792788, 0.003534366, 0.0027776486, 0.0026..."
2,2,"[administrative_court, military_service, supre...","[0.014369766, 0.010394671, 0.005361035, 0.0053..."
3,3,"[breach_peace, statistical_data, access_offici...","[0.0031214887, 0.001970042, 0.0016735342, 0.00..."
4,4,"[applicant_company, radio_television, national...","[0.022619745, 0.006330287, 0.0048440224, 0.004..."
5,5,"[penal_code, section_penal, commit_suicide, th...","[0.0013777969, 0.0012725499, 0.0010093744, 0.0..."
6,6,"[applicant_company, supreme_court, penal_code,...","[0.0078093857, 0.004838982, 0.00316657, 0.0024..."
7,7,"[court_appeal, personal_data, search_engine, r...","[0.008935143, 0.0062825386, 0.0043751593, 0.00..."
8,8,"[court_cassation, article_code, president_cour...","[0.007679069, 0.0051480443, 0.0036496827, 0.00..."
9,9,"[first_applicant, publish_advertisement, supre...","[0.0033783664, 0.0031147255, 0.0028985357, 0.0..."


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence score: https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know